In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [11]:
response_schemas

[ResponseSchema(name='answer', description='사용자의 질문에 대한 답변', type='string'),
 ResponseSchema(name='source', description='사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.', type='string')]

In [12]:
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='answer', description='사용자의 질문에 대한 답변', type='string'), ResponseSchema(name='source', description='사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.', type='string')])

In [4]:
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [13]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // 사용자의 질문에 대한 답변\n\t"source": string  // 사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.\n}\n```'

In [14]:
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // 사용자의 질문에 대한 답변\n\t"source": string  // 사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.\n}\n```'}, template='answer the users question as best as possible.\n{format_instructions}\n{question}')

In [5]:
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                                temperature=0,
                                max_tokens=None,
                                timeout=None,) # 모델 초기화
chain = prompt | model | output_parser  # 프롬프트, 모델, 출력 파서를 연결

In [15]:
model

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000199EF5BD310>, default_metadata=(), model_kwargs={})

In [16]:
chain

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // 사용자의 질문에 대한 답변\n\t"source": string  // 사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.\n}\n```'}, template='answer the users question as best as possible.\n{format_instructions}\n{question}')
| ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000199EF5BD310>, default_metadata=(), model_kwargs={})
| StructuredOutputParser(response_schemas=[ResponseSchema(name='answer', description='사용자의 질문에 대한 답변', type='string'), ResponseSchema(name='source', description='사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.', type='string')])

In [17]:
# 대한민국의 수도가 무엇인지 질문합니다.
question = "전주대학교는 어디에 있나요?"
chain.invoke({"question": question})

{'answer': '전주대학교는 전라북도 전주시 완산구 효자동 3가에 위치하고 있습니다.', 'source': '전주대학교 웹사이트'}

In [19]:
questions = [
    "전주대학교는 무슨 학교인가요?",
    "전주대학교의 위치는 어디인가요?",
    "전주대학교의 설립 연도는 언제인가요?",
    "전주대학교가 있는 도시는 어디인가요?",
]
model.batch(questions)

[AIMessage(content='전주대학교는 대한민국 전라북도 전주시에 위치한 사립 종합대학교입니다. \n\n**주요 특징:**\n\n*   **역사:** 1964년에 설립된 기독교 정신을 바탕으로 하는 대학교입니다.\n*   **캠퍼스:** 아름다운 캠퍼스와 다양한 시설을 자랑합니다.\n*   **학과:** 인문사회, 자연과학, 공학, 예체능 등 다양한 분야의 학과를 운영하고 있습니다.\n*   **특성화 분야:** 한식조리학과, 게임콘텐츠학과 등이 유명하며, 지역사회와의 연계를 강화하고 있습니다.\n*   **국제 교류:** 해외 대학과의 교류 프로그램을 통해 학생들에게 국제적인 경험을 제공합니다.\n\n더 자세한 정보는 전주대학교 공식 웹사이트([https://www.jj.ac.kr/](https://www.jj.ac.kr/))에서 확인하실 수 있습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--aa332841-e14d-453d-bc11-18bd2a9501e9-0', usage_metadata={'input_tokens': 13, 'output_tokens': 246, 'total_tokens': 259, 'input_token_details': {'cache_read': 0}}),
 AIMessage(content='전주대학교는 전라북도 전주시 완산구 효자동 3가에 위치하고 있습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gem